In [ ]:
import requests
import pandas as pd

# get the html page
url_list = ["https://www.indianhealthyrecipes.com/recipes/south-indian-recipes-food/",
            "https://www.indianhealthyrecipes.com/recipes/south-indian-recipes-food/page/2/",
            "https://www.indianhealthyrecipes.com/recipes/south-indian-recipes-food/page/3/",
            "https://www.indianhealthyrecipes.com/recipes/south-indian-recipes-food/page/4/",
            "https://www.indianhealthyrecipes.com/recipes/south-indian-recipes-food/page/5/"
            ]
custom_headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/116.0'}


In [ ]:
# find required hyperlinks
from bs4 import BeautifulSoup

recipe_links = []
recipe_data = []

for url in url_list:
    html_page = requests.get(url, headers=custom_headers)
    # print(page.text)

    soup = BeautifulSoup(html_page.text, "html.parser")
    exclusion_list = ['facebook', 'twitter', 'instagram', 'pinterest', 'youtube', 'about', '/receipes/']
    for link in soup.findAll('a'):
        page_link = link.get('href')
        # print(page_link, " = ", any(ele in link.get('href') for ele in exclusion_list))
        try:
            if ('indianhealthyrecipes' in str(page_link) and (any(ele in str(page_link)for ele in exclusion_list) is not True)):
                #  print(page_link)
                recipe_links.append(page_link)
        except:
            print("exception occured ", page_link)
# print(recipe_links)
print(len(recipe_links))

# get unique list
recipe_links = list(dict.fromkeys(recipe_links))
print(recipe_links)
print(len(recipe_links))


In [ ]:

limit = 5
for index,recipe_url in enumerate(recipe_links):

    recipe_data_row = []

    # recipe_url = 'https://www.indianhealthyrecipes.com/tomato-rasam-recipe/'
    recipe_name = recipe_url.replace('https://www.indianhealthyrecipes.com/', '').replace('/','').replace('-recipe','').replace('recipes','')
    print(recipe_name)
    
    if (recipe_name != ''):
        recipe_data_row.append(recipe_name)
        recipe_page = requests.get(recipe_url, headers=custom_headers)
        recipe_soup = BeautifulSoup(recipe_page.text, "html.parser")
        ingredients = recipe_soup.find_all('span', attrs = {"class": "wprm-recipe-ingredient-name"})
        ingredients_list = []
        for span in ingredients:
            ingredients_list.append(span.get_text())

        # get unique list
        ingredients_list = list(dict.fromkeys(ingredients_list))
        recipe_data_row.append(','.join(ingredients_list))

        prep_time = recipe_soup.find_all('span', attrs={"class": "wprm-recipe-prep_time-minutes"})
        for span in prep_time:
            prep_time_in_mins = span.get_text().replace('minutes', '').replace(' ','')
        recipe_data_row.append(prep_time_in_mins)

        cook_time = recipe_soup.find_all('span', attrs={"class": "wprm-recipe-cook_time-minutes"})
        for span in cook_time:
            cook_time_in_mins = span.get_text().replace('minutes', '').replace(' ','')
        recipe_data_row.append(cook_time_in_mins)

        recipe_data_row.append(recipe_url)
        # print(recipe_data_row)

        recipe_data.append(recipe_data_row)
        # print(recipe_data)

    # if index == limit:
    #     break


In [ ]:
import os

df = pd.DataFrame(recipe_data, columns=['recipe','ingredients','prep_time_in_mins','cook_time_in_mins','recipe_link'])
# print(df)

report_path = os.path.dirname('') + "../data/recipe-data.csv"
df.to_csv(report_path)